# Файл для отладки

![image](../pic/pic1.jpg)
![image](../pic/pic2.jpg)

In [1]:
Generate_Network_t0 <- function(size = 40, sparsity = 0.7) {
	network <- matrix(0, nrow=size, ncol=size)
  
	for (i in 1:size) {
		for (j in 1:size) {
			if (runif(1) > sparsity) {
				network[i, j] <- 1
			}
		}
	}
  
	return(network)
}


Network_function_1 <- function(i, Network) { # outdegree (density)
	net <- sum(Network[i, ])
	return(net)
}


Network_function_2 <- function(i, Network) { # recip
	net <- 0
    for(j in 1:length(Network[1,])){
        if(i != j){
            net <- net + Network[i,j]*Network[j,i]
        }
    }
	return(net)
}

Network_function_3 <- function(i, Network) { # transitive triads (transTrip)
	net <- 0
	for (j in 1:length(Network[i, ])) {
		for(h in 1:length(Network[i, ])){
			if(i!=j & j!=h & i!=h){
				net <- net + (Network[i,j] * Network[i,h] * Network[h,j]) 
			}
		}
	}
	return(net)
}

Generate_Possible_Ministep_Matrix_vector <- function(i, Network) {
	Networks <- list()
  
	for (j in 1:length(Network[i, ])) {
		if (i != j) {
			TMP_Net <- Network
			TMP_Net[i, j] <- 1
			Networks[[j]] <- TMP_Net    
		}
	}
	return(Networks)
}

Ministep <- function(Network, i, j) {
	Network[i, j] <- 1
	return(Network)
}


Distribution_actors <- function(i, Network, beta) {
	Util <- numeric( length(Network[i, ]) )
	total_util <- 0
  
	for (h in 1:length(Network[i, ])) {
		if (i != h) {
			total_util <- total_util + exp(Utility(Network, beta, i, h))
		}
	}
  
	Util_j <- numeric(length(Network[i, ]))
  
	for (j in 1:length(Network[i, ])) {
		if (j != i) {
			Util_j[j] <- exp(Utility(Network, beta, i, j))
			Util[j] <- Util_j[j] / total_util
		}
        
	}
    Util[which(is.na(Util))] <- 0
	return(Util)
}

Utility <- function(Network, beta, i, j) {
	util <- 0
    NetIJ <- Generate_Possible_Ministep_Matrix_vector(i, Network)
	util <- util + (beta[1] * Network_function_2(i, NetIJ[[j]])) + (beta[2] * Network_function_3(i, NetIJ[[j]]))
	return(util)
}

Simulator <- function(Network0, t = 0, LambdaV, total_Lambda, T, beta) {
	Network <- Network0 
	while (t < T) {
		deltaT <- rexp(1, rate=total_Lambda)
		i <- sample(1:length(Network0[1,]), size=1, prob=(LambdaV / total_Lambda))
      
		prob_vector <- 0
		prob_vector <- Distribution_actors(i, Network, beta)
		j <- sample(1:length(Network0[i,j]), size=1, prob=prob_vector)
		Network[i, j] <- 1
		t <- t + deltaT
        

	}
	return(Network)
}

In [2]:
size <- 5
Network0 <- Generate_Network_t0(size = size, sparsity = .7)
# LambdaV <- rexp(n = size)
LambdaV <- runif(n = size, min = 0, max = 1)
total_Lambda <- sum(LambdaV)
beta <- rexp(n = 2)
# beta <- c(1.5, 1.8)
T <- 2.5

In [3]:
Network0

LambdaV

total_Lambda

beta

LambdaV / total_Lambda



1,0,0,0,0
0,0,1,0,0
1,0,1,0,0
0,0,0,1,0
0,0,1,1,0


[1] 0.1254742 0.6603874 0.3826041 0.7296405 0.9443434

[1] 2.84245

[1] 1.2691526 0.2242439

[1] 0.04414298 0.23233038 0.13460366 0.25669427 0.33222872

# Эффекты верно работают

In [4]:
for(i in 1:size){
    print(Network_function_1(i = i,Network = Network0))
}

[1] 1
[1] 1
[1] 2
[1] 1
[1] 2


In [5]:
for(i in 1:size){
    print(Network_function_2(i = i,Network = Network0))
}

[1] 0
[1] 0
[1] 0
[1] 0
[1] 0


In [6]:
for(i in 1:size){
    print(Network_function_3(i = i,Network = Network0))
}

[1] 0
[1] 0
[1] 0
[1] 0
[1] 0


# Распределения

In [7]:
for (i in 1:size){
    print(Distribution_actors(i, Network0, beta))
    print(sum(Distribution_actors(i, Network0, beta)))
}


[1] 0.0000000 0.1524893 0.5425320 0.1524893 0.1524893
[1] 1
[1] 0.2779136 0.0000000 0.2220864 0.2220864 0.2779136
[1] 1
[1] 0.1097012 0.3902988 0.0000000 0.1097012 0.3902988
[1] 1
[1] 0.1524893 0.1524893 0.1524893 0.0000000 0.5425320
[1] 1
[1] 0.2779136 0.2779136 0.2220864 0.2220864 0.0000000
[1] 1


In [10]:
U <- matrix(ncol = size, nrow = size, data = 0)
for(i in 1:size){
    for(j in 1:size){
        if(i!= j){
            U[i,j] <- Utility(Network0, beta, i,j)    
        }
        else {U[i,j] <- 0}
        }
    }
U

0.0000000,0.0000000,1.269153,0,0.0000000
0.2242439,0.0000000,0.000000,0,0.2242439
0.0000000,1.2691526,0.000000,0,1.2691526
0.0000000,0.0000000,0.000000,0,1.2691526
0.2242439,0.2242439,0.000000,0,0.0000000
